<a href="https://colab.research.google.com/github/mitsu-h/BirdCLEF/blob/oversampling/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install audiomentations

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
from datetime import datetime
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

import torchaudio
from torchaudio import transforms
import torchvision
from audiomentations import Compose, AddGaussianNoise, PitchShift, TimeStretch, Shift

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
def torch_fix_seed(seed=42):
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


torch_fix_seed()

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_dir="/content/drive/MyDrive/colab/BirdCLEF/"
data_dir= os.path.join(root_dir, "inputs/")
model_dir = os.path.join(root_dir, "models/")

In [ ]:
train_df = pd.read_csv(os.path.join(data_dir, 'train_metadata.csv'))
train_df.head()

# Quick EDA and Data Pre-processing

In [ ]:
train_df.shape

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

Looks like no null values are there.

In [ ]:
# For this baseline notebook, we would consider following columns 
columns_of_interest = ['primary_label', 'secondary_labels', 'rating', 'filename']
train_df = train_df[columns_of_interest]
train_df.head()

In [ ]:
f, ax = plt.subplots(figsize=(20,8))
sns.countplot(x='rating', data=train_df, ax=ax)

we will consider only those audio files whose rating >= 3.0

In [ ]:
train_df = train_df[train_df.rating >= 3.0]

primary_labelのクラス比


In [ ]:
f, ax = plt.subplots(figsize=(30,8))
sns.countplot(x="primary_label", data=train_df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

secondary_labelsのクラス比

In [ ]:
secondary_labels = sum(train_df["secondary_labels"].apply(lambda x: eval(x)).values, [])
len(set(secondary_labels))

In [ ]:
f, ax = plt.subplots(figsize=(30,8))
sns.countplot(x=sorted(secondary_labels), ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

### データが不均衡なため、データ数の逆数を重みとして利用

In [ ]:
label_weights = 1. / train_df.groupby("primary_label").count()["filename"]
label_weights

In [ ]:
train_df.groupby("primary_label").count().describe()

In [ ]:
train_df = train_df.reset_index(drop=True)
train_df.head()

### label情報の作成

In [ ]:
labels = train_df.primary_label.unique().tolist()

In [ ]:
label2id = {labels[i]: i for i in range(len(labels))}

In [ ]:
labels[0], label2id['afrsil1']

In [ ]:
train_audio_dir = os.path.join(data_dir, 'train_audio')

In [ ]:
# for i in tqdm(range(len(train_df))):
#     filename = train_df.loc[i,'filename']
#     sig, sr = torchaudio.load(os.path.join(train_audio_dir, filename))
#     train_df.loc[i, 'num_channels'] = sig.shape[0]  # number of audio channels (mono/stereo)
#     train_df.loc[i, 'signal_len'] = sig.shape[1]  # signal length
#     train_df.loc[i, 'sampling_rate'] = sr

In [ ]:
# train_df['num_channels'] = train_df['num_channels'].astype('int64')
# train_df['signal_len'] = train_df['signal_len'].astype('int64')
# train_df['sampling_rate'] = train_df['sampling_rate'].astype('int64')
# train_df.head()

In [ ]:
# train_df.sampling_rate.unique()

we have unqiue sampling rate i.e., 32 KHz

For a 1 second video, the array size will be 32000

In [ ]:
# sns.countplot(train_df.num_channels)

There are more number of two channel audios. So we will convert mono (1 channel ) audio to stereo (2 channel) audio by replication.

In [ ]:
def MonoToStereo(aud, num_channel=2):
    sig, sr = aud
    if sig.shape[0] == num_channel:
        return aud
    else:
        stereo_sig = torch.cat([sig, sig])
    
    return (stereo_sig, sr)

In [ ]:
# train_df.signal_len.min(), train_df.signal_len.max()

In [ ]:
# f, ax = plt.subplots(figsize=(15,8))
# sns.distplot(train_df.signal_len, ax=ax)

we need to have signal length to be of same size. So we will either pad the signal or truncate the signal.

In [ ]:
# Let's consider the length of all videos to be 10 seconds (A hyperparam - to be tuned)
max_len_ms = 10000 # 10k milliseconds ~ 10 seconds

def pad_signal(aud, max_len_ms):
    sig, sr = aud
    num_channels, sig_len = sig.shape
    max_len = sr // 1000 * max_len_ms
    
    if sig_len > max_len:
        sig = sig[:, :max_len]
    elif sig_len < max_len:
        # padding both sides of the signal
        pad_begin_len = random.randint(0, max_len - sig_len)
        pad_end_len = max_len - sig_len - pad_begin_len
        
        # pad with zeros
        pad_begin = torch.zeros((num_channels, pad_begin_len))
        pad_end = torch.zeros((num_channels, pad_end_len))
        
        sig = torch.cat((pad_begin, sig, pad_end), 1)
    
    return (sig, sr)

# Data Augmentation on raw audio
下記リンクから実装内容を選択<br>
https://github.com/iver56/audiomentations/tree/master/audiomentations/augmentations

In [ ]:
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    #TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    #PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    #Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5), 
])

In [ ]:
# Convert augmented audio to Mel Spectrogram
def mel_spec(aud, n_mels=128, n_fft=None, hop_len=None):
    sig, sr = aug_audio
    top_db = 80
    n_fft = n_fft or sr // 10
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    # shape of spec: (channels, n_mels, time)
    
    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    
    # Add channel
    spec = torch.cat([spec, spec.mean(dim=0, keepdim=True)])
    return spec

In [ ]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)
    
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = torch.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.to(torch.uint8)
    else:
        V = torch.zeros_like(X, dtype=torch.uint8)

    return V

def normalize(image):
        image = image.to(torch.float32) / 255.0
        return image  


In [ ]:
# Data augmentation on mel spectrogram: Time and Frequency Masking
def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec
    
    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
        aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)
        
    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
        aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
        
    return aug_spec

Preprocessing on one signal sample

In [ ]:
idx = random.randint(0, len(train_df))
filename = train_df.loc[idx,'filename']
sig, sr = torchaudio.load(os.path.join(train_audio_dir, filename))
audio_sample = (sig, sr)

# mono to stereo (if mono)
audio = MonoToStereo(audio_sample)

# Pad or Truncate
audio = pad_signal(audio, max_len_ms)

# Augment on raw audio
# aug_audio = augment(*audio)
# aug_audio = (aug_audio, sr)
aug_audio = audio

# Convert to Mel Spectrogram
spec = mono_to_color(mel_spec(aug_audio))

# Normarize
spec = normalize(spec)

# Augment on mel spec
aug_spec =  spec #spectro_augment(spec)
aug_spec.shape

In [ ]:
aug_spec_np = aug_spec.permute(1,2,0).numpy()
f, ax = plt.subplots(figsize=(15,8))
plt.imshow(aug_spec_np[:, :, 0])
plt.colorbar()
plt.show()
f, ax = plt.subplots(figsize=(15,8))
plt.imshow(aug_spec_np[:, :, 1])
plt.show()
f, ax = plt.subplots(figsize=(15,8))
plt.imshow(aug_spec_np[:, :, 1])
plt.show()

In [ ]:
from IPython.display import Audio, display
def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

In [ ]:
play_audio(sig, sr)

# Building Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, audio_dir, df, labels, max_len_ms=10000, shift_limit=0.4, input_size=224):
        self.train_audio_dir = audio_dir
        self.train_df = df
        self.max_len_ms = max_len_ms
        self.shift_limit = shift_limit
        self.input_size = input_size
        self.num_classes = len(labels)
        
    def __len__(self):
        return len(self.train_df)

    def __getitem__(self, index):
        filename = self.train_df.loc[index, 'filename']
        labels = [self.train_df.loc[index, 'primary_label']]
        labels.extend(eval(self.train_df.loc[index, 'secondary_labels']))
        labels = list(set(labels))
        
        sig, sr = torchaudio.load(os.path.join(self.train_audio_dir, filename))
        audio = (sig, sr)
        
        ##################        
        # process signal #
        ##################
        
        # mono to stereo (if mono)
        audio = MonoToStereo(audio)

        # Pad or Truncate
        audio = pad_signal(audio, self.max_len_ms)

        # Augment raw audio
        # aug_audio = augment(*audio)
        # aug_audio = (aug_audio, sr)
        aug_audio = audio

        # Convert to Mel Spectrogram
        spec = mono_to_color(mel_spec(aug_audio))

        # Normarize
        spec = normalize(spec)

        # Augment mel spec
        # aug_spec = spectro_augment(spec)
        aug_spec=spec

        # make multilabel data
        y = torch.tensor([label2id[label] for label in labels])
        y_onehot = nn.functional.one_hot(y, num_classes=self.num_classes).sum(dim=0).float()

        # batch weight
        # 現状、精度に寄与しないのでコメントアウト
        # weights = torch.clone(y_onehot)
        # weights.apply_(lambda x:100 if x==1 else 1)
        
        return aug_spec, y_onehot #, weights

In [ ]:
max_len_ms = 10000
shift_limit = 0.4
input_size = 224
dataset = CustomDataset(train_audio_dir, train_df, labels, max_len_ms, shift_limit, input_size)

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

### 不均衡データの問題解消のため、samplerを利用してoversamplingを行う

In [ ]:
1./label_weights.loc['brnowl']

In [ ]:
train_df_weights = train_df.copy()
train_df_weights['weight'] = [label_weights[label] for label in train_df['primary_label']]
train_df_weights[train_df_weights['primary_label']=='brnowl']

In [ ]:
samples_weight = train_df_weights['weight'].values
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, 5000)

samplerの利用によるクラス比の違い

In [ ]:
f, ax = plt.subplots(figsize=(30,8))
sns.countplot(x="primary_label", data=train_df, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

sampler_test = list(sampler)
f, ax = plt.subplots(figsize=(30,8))
sns.countplot(x="primary_label", data=train_df.loc[sampler_test,:].sort_values('primary_label'), ax=ax)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()
# train_df.loc[sampler_test,'primary_label'].groupby('primary_label')

In [ ]:
BATCH_SIZE = 64
# DataLoaderのnum_workersは、前処理部分の並列実行を可能にしてくれる
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
# train_loader = DataLoader(train_dataset, sampler=ImbalancedDatasetSampler(train_dataset), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
train_loader = DataLoader(dataset, sampler=sampler, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
inputs, targets = next(iter(train_loader))
print(inputs.shape, targets.shape)
# print(targets[0], weights[0])

Cool!!

Let's build model architecture.

# Model Building

### model作成の関数
基本的に、モデルを変更する場合はここの修正を行う！

In [ ]:
def create_model(labels, device):
  model = torchvision.models.resnet50()
  model.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=model.fc.in_features,
        out_features=len(labels)
    ),
    torch.nn.Sigmoid()
  )
  return model.to(device)

### modelのチェック

In [ ]:
model = create_model(labels, 'cpu')
model

In [ ]:
model.eval()
output = model(inputs)
output.shape

In [ ]:
LERANING_RATE = 1e-2

# Defining loss and optimizer
criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=LERANING_RATE)
optimizer = torch.optim.SGD(model.parameters(), lr=LERANING_RATE, momentum=0.9)

# Model Training

In [ ]:
def create_confusion_matrix(y_true, y_pred):
  cf_matrix = confusion_matrix(y_true, y_pred)
  class_id  = np.unique(np.concatenate([y_true,y_pred]))
  df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix)*10, index = class_id, columns=class_id)
  plt.figure(figsize=(12,7))
  return sns.heatmap(df_cm).get_figure()

In [ ]:
# def create_roc_curve(y_true, y_pred):
#   fprs, tprs = [], []
#   for i in range(y_true.shape[1]):
#     fpr, tpr, _ = roc_curve(y_true[:,i], y_pred[:,i])

In [ ]:
val_df = train_df.loc[val_dataset.indices,['primary_label','secondary_labels']]
val_df['secondary_labels'] = val_df['secondary_labels'].apply(lambda x:eval(x))
pri = val_df['primary_label'].values
for sec in val_df['secondary_labels']:
  pri = np.append(pri, sec)
len(np.unique(pri))


In [ ]:
def train_epoch(model, data_loader, device, criterion, optimizer, num_epoch, writer):
    model.train()

    losses = []
    y_pred = None
    y_true = None
    auc = 0
    total = 0

    for batch_idx, (x, y) in enumerate(tqdm(data_loader), num_epoch*len(data_loader)):
        x = x.to(device) 
        y = y.to(device) 
        # weights = weights.to(device)

        output = model(x)

        loss = criterion(output, y) #nn.functional.binary_cross_entropy(output, y, weights) 

        writer.add_scalar('Loss/train_iter', loss.item(), batch_idx)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # _, preds = torch.max(output, dim=1)
        # preds = preds.cpu()
        output = output.detach().cpu()
        y = y.cpu()
        y_pred = np.concatenate([y_pred, output.numpy()]) if y_pred is not None else output.numpy()
        y_true = np.concatenate([y_true, y.numpy()]) if y_true is not None else y.numpy()
        # auc += roc_auc_score(y.numpy(), output.detach().cpu().numpy())
        # total += y.size(0)

    # writer.add_figure("confusion matrix/train", create_confusion_matrix(y_true, y_pred),num_epoch)
    # print(f'y_pred:{y_pred}, y_true:{y_true}')
    auc = roc_auc_score(y_true, y_pred)

    return auc, np.mean(losses)

In [ ]:
def val_epoch(model, data_loader, device, criterion):
    model.eval()

    losses = []
    y_pred = None
    y_true = None
    auc = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(tqdm(data_loader)):
            x = x.to(device) 
            y = y.to(device) 

            output = model(x)

            loss = criterion(output, y)

            losses.append(loss.item())

            # _, preds = torch.max(output, dim=1)
            # preds = preds.cpu()
            output = output.detach().cpu()
            y = y.cpu()
            y_pred = np.concatenate([y_pred, output.numpy()]) if y_pred is not None else output.numpy()
            y_true = np.concatenate([y_true, y.numpy()]) if y_true is not None else y.numpy()
            # auc += roc_auc_score(y.numpy(), output.detach().cpu().numpy())
            # total += y.size(0)

    # writer.add_figure("confusion matrix/train", create_confusion_matrix(y_true, y_pred),num_epoch)
    auc = roc_auc_score(y_true, y_pred)

    return auc, np.mean(losses)

In [ ]:
def train(model, epochs, device, train_loader, val_loader, criterion, optimizer, model_name, log_dir=None, comment=''):
    # logの配置ディレクトリを変えたい場合はlog_dirを引数として渡す
    # log自体に名前を付けて差別化したいときは、commentに適当な文字列を引数として渡す
    writer = SummaryWriter(log_dir=log_dir, comment=comment)

    best_val_auc = 0

    for epoch in range(epochs):
        print(f'Epoch: {epoch + 1}/{epochs}')
        print('-' * 10)
        print('Training')
        train_auc, train_loss = train_epoch(model, train_loader, device, criterion, optimizer, epoch, writer)
        # AUCの計算上、クラス数が0 or 1のみのデータがあるとエラーが出るので、一旦valはなし
        # print('\nValidating')
        # val_auc, val_loss = val_epoch(model, val_loader, device, criterion)

        print(f'\nTrain Loss: {train_loss}\tTrain AUC: {train_auc}')
        # print(f'Val Loss: {val_loss}\tVal AUC: {val_auc}')

        writer.add_scalar('Loss/train',train_loss, epoch)
        writer.add_scalar('AUC/train',train_auc, epoch)
        # writer.add_scalar('Loss/val',val_loss, epoch)
        # writer.add_scalar('AUC/val',val_auc, epoch)
        # writer.add_figure('confusion matrix/val', cf_matrix, epoch)

        if train_auc > best_val_auc:
            best_val_acc = train_auc
            torch.save(model.state_dict(), model_name)

In [ ]:
EPOCHS = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# modelの上書きを防ぐため、指定した名前or日付をつける
suffix = None # 日付以外の名前を付けたい場合はここを適当な文字列に変更
suffix = suffix or datetime.now().strftime("%Y%m%d%H%M")
model_name = os.path.join(model_dir, f"best_model_{suffix}.pth.tar")

In [ ]:
log_dir = os.path.join(root_dir, 'runs')
%tensorboard --logdir=$log_dir

In [ ]:
model = model.to(DEVICE)
train(model, EPOCHS, DEVICE, train_loader, val_loader, criterion, optimizer, model_name, log_dir=log_dir)

# Model Testing

In [ ]:
# 特定のモデルの出力を見たい場合は、ここでモデルのパスを指定する
# model_name = '/content/drive/MyDrive/colab/BirdCLEF/models/best_model_202204140824.pth.tar'
model.load_state_dict(torch.load(model_name))

In [ ]:
output = model(inputs.to(DEVICE))
output[1], targets[1]

In [ ]:
targets[1].argmax()

In [ ]:
output[1,102], targets[1,102]

In [ ]:
labels[92]